In [2]:
import os
import json
import math
import numpy as np
import pandas as pd

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()


## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import FashionMNIST
from torchvision import transforms

/tmp/ipykernel_3052/1133031499.py:11: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda


In [4]:
train_transform = transforms.Compose(
    [transforms.ToTensor()])

dataset = torchvision.datasets.ImageFolder(root ="data/", transform=train_transform)

train_loader = data.DataLoader(dataset, batch_size=256, shuffle=True, drop_last=True, pin_memory=True)

def get_train_images(num):
    return torch.stack([dataset[i][0] for i in range(10,10+num)], dim=0)

In [5]:
dataset.samples

[('data/00000/00000_00000/00000_00000.jpg', 0),
 ('data/00000/00000_00001/00000_00001.jpg', 0),
 ('data/00000/00000_00002/00000_00002.jpg', 0),
 ('data/00000/00000_00003/00000_00003.jpg', 0),
 ('data/00000/00000_00004/00000_00004.jpg', 0),
 ('data/00000/00000_00005/00000_00005.jpg', 0),
 ('data/00000/00000_00006/00000_00006.jpg', 0),
 ('data/00000/00000_00007/00000_00007.jpg', 0),
 ('data/00000/00000_00008/00000_00008.jpg', 0),
 ('data/00000/00000_00009/00000_00009.jpg', 0),
 ('data/00000/00000_00010/00000_00010.jpg', 0),
 ('data/00000/00000_00011/00000_00011.jpg', 0),
 ('data/00000/00000_00012/00000_00012.jpg', 0),
 ('data/00000/00000_00013/00000_00013.jpg', 0),
 ('data/00000/00000_00014/00000_00014.jpg', 0),
 ('data/00000/00000_00015/00000_00015.jpg', 0),
 ('data/00000/00000_00016/00000_00016.jpg', 0),
 ('data/00000/00000_00017/00000_00017.jpg', 0),
 ('data/00000/00000_00018/00000_00018.jpg', 0),
 ('data/00000/00000_00019/00000_00019.jpg', 0),
 ('data/00000/00000_00020/00000_00020.jp

In [5]:
get_train_images(10)

tensor([[[[0.6118, 0.7451, 0.6784,  ..., 0.4824, 0.3137, 0.0000],
          [0.7255, 0.6314, 0.6627,  ..., 0.5804, 0.3804, 0.0118],
          [0.7412, 0.6039, 0.7098,  ..., 0.5843, 0.4078, 0.0549],
          ...,
          [0.6196, 0.6235, 0.5686,  ..., 0.2549, 0.2941, 0.0157],
          [0.6314, 0.8510, 0.8353,  ..., 0.2392, 0.2784, 0.0039],
          [0.6157, 0.9176, 0.9333,  ..., 0.2392, 0.2667, 0.0039]],

         [[0.4431, 0.4275, 0.2157,  ..., 0.3647, 0.3059, 0.0118],
          [0.4902, 0.2824, 0.2078,  ..., 0.4157, 0.3176, 0.0000],
          [0.4078, 0.2000, 0.2549,  ..., 0.4039, 0.3098, 0.0000],
          ...,
          [0.4235, 0.4667, 0.4784,  ..., 0.2784, 0.3020, 0.0000],
          [0.4431, 0.6941, 0.7216,  ..., 0.2745, 0.2980, 0.0000],
          [0.4784, 0.7843, 0.8235,  ..., 0.2824, 0.2902, 0.0000]],

         [[0.4078, 0.4118, 0.2157,  ..., 0.3569, 0.3490, 0.0902],
          [0.4471, 0.2588, 0.2118,  ..., 0.3686, 0.3176, 0.0275],
          [0.3294, 0.1647, 0.2588,  ..., 0

In [ ]:
def generate_images(model, n_imgs, device):
    # Generate images
    model.eval()
    with torch.no_grad():
        generated_imgs = model.decoder(torch.randn([n_imgs, model.latent_dim]).to(device))
    generated_imgs = generated_imgs.cpu()

    grid = torchvision.utils.make_grid(generated_imgs, nrow=4, normalize=False)
    grid = grid.permute(1, 2, 0)
    plt.figure(figsize=(15,10))
    plt.title(f"Generations")
    plt.imshow(grid)
    plt.axis('off')
    plt.show()

In [ ]:
######### Frechet Inception distance based on implementation from https://github.com/mseitzer/pytorch-fid

import numpy as np
from scipy import linalg


def calculate_frechet_distance(distribution_1, distribution_2, eps=1e-6):
    mu1 = np.mean(distribution_1, axis=0)
    sigma1 = np.cov(distribution_1, rowvar=False)

    mu2 = np.mean(distribution_2, axis=0)
    sigma2 = np.cov(distribution_2, rowvar=False)

    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
    Stable version by Dougal J. Sutherland.
    Params:
    -- mu1   : Numpy array containing the activations of a layer of the
               inception net (like returned by the function 'get_predictions')
               for generated samples.
    -- mu2   : The sample mean over activations, precalculated on an
               representative data set.
    -- sigma1: The covariance matrix over activations for generated samples.
    -- sigma2: The covariance matrix over activations, precalculated on an
               representative data set.
    Returns:
    --   : The Frechet Distance.
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    # Product might be almost singular
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # Numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)

# VAE

# GAN

# GLOW

# VAEGAN

https://github.com/lucabergamini/VAEGAN-PYTORCH